In [1]:

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


The function get_10_K() below takes 2 parameters. 
        # >"year" is the year of 10-K report.
        # >"start" is the corresponding iterator that iterates through 1->the number of 10-K reports

In [2]:

def get_10_K(start,year="2018",path='/gcloud/'):
# initiate the url
    start=str(start)
    url_1="https://www.sec.gov/cgi-bin/srch-edgar?text=%2210-K%22%20NOT%20%22NT%22&start="+start+"&count=1&first="+year+"&last="+year

#Step 1: access the search website
    response = requests.get(url_1)
    if response.status_code == 200:
        pass
    else:
        print("Failure")

#Step 2: Print and Find the 10K report tag
    results_page = BeautifulSoup(response.content,'lxml')
    #print(results_page.prettify())

    td_tag = results_page.find('td',align="left",valign="top")

# Generate the URL for next step.
    CompanyName=td_tag.find("a").get_text()
    url_2=td_tag.find("a").get("href")
    url_2="https://www.sec.gov"+url_2

# Step 3: access the index file
    response_2 = requests.get(url_2)
    if response_2.status_code == 200:
        pass
    else:
        print("Failure")
# Step 4: genreate the url of the 10-K report: url_3
    results_page_2 = BeautifulSoup(response_2.content,'lxml')
    #print(results_page_2.prettify())

    all_td_tags_2 = results_page_2.find_all('td',scope="row")

    url_3=all_td_tags_2[2].find("a").get("href")
    url_3="https://www.sec.gov"+url_3

    response_3=requests.get(url_3)
    if response_3.status_code == 200:
        pass
    else:
        print("Failure")

    results_page_3 = BeautifulSoup(response_3.content,'lxml')
    #print(results_page.prettify())
# Step 5: search for and  gather the text file( assume all useful information is embedded in tab <p> )
    all_font_tags = results_page_3.find_all('p')

    text=""
    for tag in all_font_tags:
        text=text+" "+tag.get_text()

# Step 6: If fails, try assumption: tab <font> )
    if len(text)<5000:
        all_font_tags = results_page_3.find_all('font')

        text=""
        for tag in all_font_tags:
            text=text+" "+tag.get_text()
# Step 7: If still fails, skip the document-writing part and print error message)   
    flag=(len(text)<8000)
    
    if not os.path.exists(path):
        os.makedirs(path)
    if not flag:
        CompanyName=CompanyName.replace("/","_")
        CompanyName=CompanyName.replace("\\","_")
        with open(path+CompanyName+'.txt', 'w',encoding="utf-8") as f:  
            f.write(text)
            f.close()
    else:
        print("Company Name:",CompanyName)
        print("Text Length",len(text))
        print("link to 10-K",url_3)
    return flag,CompanyName,len(text),url_3

Follow the template below to write all 10-K reports!

In [4]:
Failed_dict={}
for start in tqdm(range(1,7001)):
    flag,CompanyName,text_len,url=get_10_K(start)
    
    if flag:
        print(start,"Fail!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        Failed_dict[CompanyName]=url
with open(path+'Failed_dict.txt', 'w',encoding="utf-8") as f:  
    f.write(Failed_dict)
    f.close()

NameError: name 'tqdm' is not defined